# 10.1 GroupBy机制

## 10.1.1 遍历各分组

In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
df = pd.DataFrame({'key1':list('aabba'),
                   'key2':['one', 'two', 'one', 'two', 'one'],
                   'data1':np.random.randn(5),
                   'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.930427,0.536521
1,a,two,-2.223075,-0.644195
2,b,one,-2.325548,-0.175350
3,b,two,-0.097791,-1.016030
4,a,one,0.892176,-1.007044


In [5]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.930427  0.536521
1    a  two -2.223075 -0.644195
4    a  one  0.892176 -1.007044
b
  key1 key2     data1    data2
2    b  one -2.325548 -0.17535
3    b  two -0.097791 -1.01603


In [7]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.930427  0.536521
4    a  one  0.892176 -1.007044
('a', 'two')
  key1 key2     data1     data2
1    a  two -2.223075 -0.644195
('b', 'one')
  key1 key2     data1    data2
2    b  one -2.325548 -0.17535
('b', 'two')
  key1 key2     data1    data2
3    b  two -0.097791 -1.01603


In [14]:
pieces = dict(list(df.groupby('key1')))
print(pieces['a'])
print(pieces['b'])

  key1 key2     data1     data2
0    a  one -0.930427  0.536521
1    a  two -2.223075 -0.644195
4    a  one  0.892176 -1.007044
  key1 key2     data1    data2
2    b  one -2.325548 -0.17535
3    b  two -0.097791 -1.01603


In [15]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [18]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.930427  0.536521
1 -2.223075 -0.644195
2 -2.325548 -0.175350
3 -0.097791 -1.016030
4  0.892176 -1.007044
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## 10.1.2 选择一列或所有列的子集

In [21]:
df.groupby('key1')['data1']

In [26]:
df.groupby(['key1', 'key2'])[['data1']].mean()

data1
key1 key2          
a    one  -0.019125
     two  -2.223075
b    one  -2.325548
     two  -0.097791

## 10.1.3 使用字典和Series分组

In [28]:
people = pd.DataFrame(np.random.randn(5,5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.293706,0.253187,3.085450,0.766237,-1.914126
Steve,-1.209777,0.327604,0.089063,-0.012180,0.432815
Wes,-0.083065,2.054734,-0.222118,0.288996,1.013641
Jim,1.510044,-1.786005,-0.738717,0.852256,-0.597125
Travis,-0.609300,-0.137751,-0.091378,-1.409919,-0.476333


In [31]:
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,0.293706,0.253187,3.085450,0.766237,-1.914126
Steve,-1.209777,0.327604,0.089063,-0.012180,0.432815
Wes,-0.083065,NaN,NaN,0.288996,1.013641
Jim,1.510044,-1.786005,-0.738717,0.852256,-0.597125
Travis,-0.609300,-0.137751,-0.091378,-1.409919,-0.476333


In [33]:
mapping = {'a':'red',
           'b':'red',
           'c':'blue',
           'd':'blue',
           'e':'red',
           'f':'orange'}
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,3.851687,-1.367233
Steve,0.076883,-0.449358
Wes,0.288996,0.930576
Jim,0.113539,-0.873086
Travis,-1.501297,-1.223384


In [35]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [36]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## 10.1.4 使用函数分组

In [37]:
people.groupby(len).count()

,a,b,c,d,e
3,3,2,2,3,3
5,1,1,1,1,1
6,1,1,1,1,1


In [39]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.083065  0.253187  3.085450  0.288996 -1.914126
  two  1.510044 -1.786005 -0.738717  0.852256 -0.597125
5 one -1.209777  0.327604  0.089063 -0.012180  0.432815
6 two -0.609300 -0.137751 -0.091378 -1.409919 -0.476333

## 10.1.5 根据索引层级分组

In [40]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      1.070415  0.137747  0.060678 -1.234092  0.397606
1      0.895823 -1.119504  0.582301 -1.762154 -0.190102
2      2.057846  0.474713  1.043976 -0.139328 -0.074876
3      1.581586 -0.402596  0.072501 -0.885881 -0.337566

In [42]:
hier_df.groupby(level='city', axis=1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 10.2 数据聚合

In [43]:
df

,key1,key2,data1,data2
0,a,one,-0.930427,0.536521
1,a,two,-2.223075,-0.644195
2,b,one,-2.325548,-0.175350
3,b,two,-0.097791,-1.016030
4,a,one,0.892176,-1.007044


In [44]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.527655
b   -0.320567
Name: data1, dtype: float64

In [49]:
df['data1'].quantile(0.2)

-2.2435699955974364

In [51]:
# 要用自己的聚合函数，需要将函数传递给 aggregate 或 agg 方法
def peak_to_peak(arr):
    return arr.max()-arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,3.115251,1.543565
b,2.227758,0.840681


In [52]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.753775  1.565121 -2.223075 -1.576751 -0.930427 -0.019125   
b      2.0 -1.211670  1.575262 -2.325548 -1.768609 -1.211670 -0.654730   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.892176   3.0 -0.371573  0.807087 -1.007044 -0.825619 -0.644195   
b    -0.097791   2.0 -0.595690  0.594451 -1.016030 -0.805860 -0.595690   

                          
           75%       max  
key1                      
a    -0.053837  0.536521  
b    -0.385520 -0.175350

## 10.2.1 逐列及多函数应用

In [54]:
tips = pd.read_csv('examples/tips.csv')
tips['tip_pct'] = tips['tip']/tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [55]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [56]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [59]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

## 10.2.2 返回不含行索引的聚合数据

In [64]:
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [67]:
tips.groupby(['day', 'smoker'])[['tip']].mean()

tip
day  smoker          
Fri  No      2.812500
     Yes     2.714000
Sat  No      3.102889
     Yes     2.875476
Sun  No      3.167895
     Yes     3.516842
Thur No      2.673778
     Yes     3.030000

In [69]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 10.3 应用：通用拆分-应用-联合

In [70]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [72]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [73]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [76]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [82]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [83]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

## 10.3.1 压缩分组键

In [89]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


## 10.3.2 分位数与桶分析


In [90]:
frame = pd.DataFrame({'data1':np.random.randn(1000),
                      'data2':np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-1.615, 0.0108]
1     (0.0108, 1.637]
2    (-3.247, -1.615]
3     (0.0108, 1.637]
4     (0.0108, 1.637]
5     (0.0108, 1.637]
6     (0.0108, 1.637]
7    (-1.615, 0.0108]
8     (0.0108, 1.637]
9     (0.0108, 1.637]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.247, -1.615] < (-1.615, 0.0108] < (0.0108, 1.637] < (1.637, 3.262]]

In [93]:
def get_stats(group):
    return{'min':group.min(),
           'max':group.max(),
           'count':group.count(),
           'mean':group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()


,count,max,mean,min
data1,,,,
"(-3.247, -1.615]",54.0,2.793094,-0.086210,-1.792328
"(-1.615, 0.0108]",484.0,4.379220,0.002218,-3.105337
"(0.0108, 1.637]",409.0,2.819494,-0.040892,-3.116692
"(1.637, 3.262]",53.0,2.118070,-0.003902,-3.570634


In [103]:
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.793094,-0.155614,-2.279272
1,100.0,2.521960,0.045384,-3.105337
2,100.0,4.379220,-0.065467,-2.186694
3,100.0,2.388308,0.018587,-2.391070
4,100.0,2.857274,0.053552,-2.223911
5,100.0,2.477238,0.154603,-2.263426
6,100.0,2.157929,-0.173456,-2.591923
7,100.0,2.732338,-0.013635,-3.116692
8,100.0,2.535984,-0.004535,-2.081069


## 10.3.3 使用指定分组值填充缺失值

In [108]:
# 使用平均值来填充NA值
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.097227
2         NaN
3   -0.501623
4         NaN
5   -1.265134
dtype: float64

In [109]:
s.fillna(s.mean())

0   -0.621328
1   -0.097227
2   -0.621328
3   -0.501623
4   -0.621328
5   -1.265134
dtype: float64

In [110]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'Califronia', 'Idaho']
group_key = ['East'] * 4 +['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          0.862329
New York     -0.835786
Vermont       0.289823
Florida      -1.395839
Oregon        0.721112
Nevada       -0.909104
Califronia   -0.443169
Idaho         1.047365
dtype: float64

In [111]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.862329
New York     -0.835786
Vermont            NaN
Florida      -1.395839
Oregon        0.721112
Nevada             NaN
Califronia   -0.443169
Idaho              NaN
dtype: float64

In [113]:
data.groupby(group_key).mean()

East   -0.456432
West    0.138971
dtype: float64

In [115]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.862329
New York     -0.835786
Vermont      -0.456432
Florida      -1.395839
Oregon        0.721112
Nevada        0.138971
Califronia   -0.443169
Idaho         0.138971
dtype: float64

In [121]:
fill_values = {'East':0.5, 'West':-1}
fill_func = lambda g:g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.862329
New York     -0.835786
Vermont       0.500000
Florida      -1.395839
Oregon        0.721112
Nevada       -1.000000
Califronia   -0.443169
Idaho        -1.000000
dtype: float64

## 10.3.4 随机采样与排列

In [124]:
# 红桃，黑桃，梅花，方块
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [125]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [126]:
# 从这副牌中拿出5张
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

JD     10
10D    10
QC     10
4D      4
8S      8
dtype: int64

In [127]:
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  4C      4
   6C      6
D  QD     10
   7D      7
H  10H    10
   3H      3
S  5S      5
   4S      4
dtype: int64

In [128]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

6C     6
2C     2
QD    10
KD    10
QH    10
6H     6
AS     1
JS    10
dtype: int64

## 10.3.5 分组加权平均和相关性

In [129]:
# 在groupby 的拆分-应用-联合的范式下， DataFrame的列间操作两个Series之间的操作，例如分组加权平均是ok的
df = pd.DataFrame({'category':['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                   'data':np.random.randn(8),
                   'weights':np.random.rand(8)})
df

,category,data,weights
0,a,-0.988198,0.889695
1,a,-0.319865,0.647340
2,a,-2.248767,0.158068
3,a,-0.002584,0.780250
4,b,0.319732,0.927249
5,b,0.915674,0.154901
6,b,1.224517,0.417229
7,b,-0.289450,0.933488


In [131]:
# 通过category进行分组加权平均如下：
grouped = df.groupby('category')
get_wavg = lambda g:np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a   -0.583242
b    0.279101
dtype: float64

In [134]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [135]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [137]:
spx_corr = lambda x:x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()
get_year = lambda x:x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


## 10.3.6 逐组线性回归

In [138]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# 10.4 数据透视表与交叉表

In [139]:
tips.pivot_table(index = ['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [140]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

## 10.4.1 交叉表

In [158]:
data = pd.DataFrame({'Sample':np.arange(1, 11),
                     'Nationality':['USA', 'Japan', 'USA', 'Japan', 'Japan', 'Japan', 'USA', 'USA', 'Japan', 'USA'],
                     'Handedness':['Right-handed', 'Left-handed', 'Right-handed', 'Right-handed', 'Left-handed', 
                                   'Right-handed', 'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed']})
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [160]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10
